# Plot comparison of costs of restricted and open designs, and resulting VoI estimate convergence

In [ ]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [ ]:
import yaml
import numpy as np
import itertools

from configs import get_experiment_config

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

#### Load design results for selected experiment

In [ ]:
expt_id = 1
settings, base_params = get_experiment_config(expt_id)
available_technologies = list(settings['probability_settings']['storage'].keys())
tech_combos = list(itertools.combinations(available_technologies, settings['model_settings']['N_technologies']))
tech_combos_strs = ['-'.join(t) for t in tech_combos]
N = settings['probability_settings']['n_prior_samples']

In [ ]:
result_dir_path_pattern = os.path.join(*settings['results_dir'],'posterior','z_scenario_{z}')

open_design_results_files = [os.path.join(result_dir_path_pattern.format(z=z), 'open_design.yaml') for z in range(N)]
open_results = [yaml.safe_load(open(f)) for f in open_design_results_files]
restricted_design_results_files = [os.path.join(result_dir_path_pattern.format(z=z), 'restricted_design.yaml') for z in range(N)]
restricted_results = [yaml.safe_load(open(f)) for f in restricted_design_results_files]

Note: comment on cost reduction from adding storage. (Compare cost to none design case)

Despite a very small proportion of the budget being spent on storage, it really reduces the cost!

A small amount of storage gives great returns on capital!

Quick bit of pre-analysis

In [ ]:
# Compute VoI
open_expected_cost = np.mean([res['overall_objective']['overall_objective'] for res in open_results])
restricted_expected_cost = np.mean([res['overall_objective']['overall_objective'] for res in restricted_results])
print(open_expected_cost/1e6)
print(restricted_expected_cost/1e6)

voi = restricted_expected_cost - open_expected_cost
print(voi/1e6)
print(voi/np.abs(restricted_expected_cost)*100)

In [ ]:
# Look at frequency of posterior optimal storage technologies
techs = ['-'.join(res['design']['storage_technologies']) for res in open_results]
counts = Counter(techs)
print(dict(counts))

#### Plot cost distributions

In [ ]:
colors = list(mcolors.TABLEAU_COLORS.values())

In [ ]:
fig,ax = plt.subplots()

plt.ylim(0,5.5e-3)

for techs in tech_combos:
    techs_str = '-'.join(techs)
    design_results_files = [os.path.join(result_dir_path_pattern.format(z=z),'design_options',f'{techs_str}_design.yaml') for z in range(N)]
    design_results = [yaml.safe_load(open(f)) for f in design_results_files]
    post_costs = np.array([res['overall_objective']['overall_objective'] for res in design_results])
    print(f"{techs_str}: {np.mean(post_costs)/1e6:.2f} €m")

    c = colors[tech_combos_strs.index(techs_str)]

    sns.kdeplot(
        post_costs/1e6,
        ax=ax,
        label=techs_str,
        color=c,
        cut=0
    )
    plt.vlines(
        np.mean(post_costs)/1e6,
        0,
        ax.get_ylim()[1],
        color=c,
        linestyle="--",
        alpha=0.5,
        zorder=1
    )

plt.xlabel('Posterior expected cost (€m)')

plt.legend()
plt.show()

Note that these pre-posterior costs are much lower than the prior costs - as expected because the system design is done with reduced uncertainty (important for budget constraint), so there is some VoI. But the interesting thing here is that given the problem structure, just doing the prior analysis leads to a significant over-estimate of expected costs! (Important for planning/financing)

This actually motivates the way of making decisions this way (i.e. adjusting capacities after R&D, cost improvement is pretty big - quantify & discuss).

#### Plot designs

In [ ]:
fig, ax = plt.subplots()

restricted_techs = restricted_results[0]['design']['storage_technologies']
restricted_techs_str = '-'.join(restricted_techs)
restricted_wind_caps = np.array([])
restricted_storage_caps = np.array([])

for res in restricted_results:
    restricted_wind_caps = np.append(restricted_wind_caps,res['design']['wind_capacity']['value'])
    total_storage_cap = 0
    for tech in restricted_techs:
        total_storage_cap += res['design']['storage_capacities'][tech]['value']
    restricted_storage_caps = np.append(restricted_storage_caps,total_storage_cap)

open_wind_caps = np.array([])
open_techs = np.array([])
open_storage_caps = np.array([])

for res in open_results:
    open_wind_caps = np.append(open_wind_caps,res['design']['wind_capacity']['value'])
    techs = res['design']['storage_technologies']
    open_techs = np.append(open_techs,'-'.join(techs))
    total_storage_cap = 0
    for tech in techs:
        total_storage_cap += res['design']['storage_capacities'][tech]['value']
    open_storage_caps = np.append(open_storage_caps,total_storage_cap)

ax.scatter(
    x=restricted_wind_caps[open_techs != restricted_techs_str]/1e6,
    y=restricted_storage_caps[open_techs != restricted_techs_str]/1e6,
    c=colors[tech_combos_strs.index(restricted_techs_str)],
    alpha=0.5,
    lw=0,
    label='Restricted',
    marker='s',
    s=15
)

for techs_str in tech_combos_strs:
    ax.scatter(
        x=open_wind_caps[open_techs == techs_str]/1e6,
        y=open_storage_caps[open_techs == techs_str]/1e6,
        c=colors[available_technologies.index(techs_str)],
        label=techs_str,
        s=12
    )

plt.xlabel('Wind Capacity (GW)')
plt.ylabel('Storage Capacity (GWh)')

lgnd = plt.legend()
for i in range(len(tech_combos)+1):
    lgnd.legendHandles[i]._sizes = [40]
plt.show()

#### Plot improvement in cost from open design c.f. restricted

In [ ]:
open_costs = np.array([])
restricted_costs = np.array([])
open_techs = np.array([])

for i in range(N):
    open_costs = np.append(open_costs,open_results[i]['overall_objective']['overall_objective'])
    restricted_costs = np.append(restricted_costs,restricted_results[i]['overall_objective']['overall_objective'])
    open_techs = np.append(open_techs,'-'.join(open_results[i]['design']['storage_technologies']))

cost_improvements = restricted_costs - open_costs

voi_estimates = np.array([np.mean(cost_improvements[:i+1]) for i in range(N)])
voi_std_errors = np.array([np.std(cost_improvements[:i+1])/np.sqrt(i+1) for i in range(N)])

In [ ]:
# plot cost comparison open vs restricted case
fig, ax = plt.subplots()

sns.histplot(
    cost_improvements/1e6,
    binrange=(0,100),
    binwidth=5,
    stat='percent',
    ax=ax,
    color='black'
)

plt.xlim(0,100)

plt.xlabel('Cost Improvement (EUR/yr, millions)')
plt.ylabel('% of Scenarios')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

ax.plot(
    np.arange(1,N+1),
    voi_estimates/1e6,
    label='VoI estimate',
    color='black',
    lw=2.5,
    zorder=1
)
ax.fill_between(
    np.arange(1,N+1),
    (voi_estimates - 2*voi_std_errors)/1e6,
    (voi_estimates + 2*voi_std_errors)/1e6,
    label='95% CI',
    color='black',
    lw=0,
    alpha=0.2,
    zorder=0
)

for techs_str in tech_combos_strs:
    ax.scatter(
        x=np.arange(1,N+1)[open_techs == techs_str],
        y=cost_improvements[open_techs == techs_str]/1e6,
        label=techs_str,
        clip_on=False,
        zorder=100
    )

plt.xlim(0,N+1)
plt.ylim(0)

plt.xlabel('Scenario')
plt.ylabel('Cost Improvement (EUR/yr, millions)')

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    np.arange(1,N+1),
    voi_estimates/1e6,
    color='black',
    lw=2.5,
    zorder=1
)
ax.fill_between(
    np.arange(1,N+1),
    (voi_estimates - 2*voi_std_errors)/1e6,
    (voi_estimates + 2*voi_std_errors)/1e6,
    color='black',
    lw=0,
    alpha=0.2,
    zorder=0
)

plt.xlim(0,N)
plt.ylim(0)

plt.xlabel('Scenario')
plt.ylabel('VoI estimate (EUR/yr, millions)')

plt.show()

Plot scenarios ordered by cost improvement

In [ ]:
ordered_cost_improvements = cost_improvements[np.argsort(cost_improvements)]
ordered_open_techs = open_techs[np.argsort(cost_improvements)]


fig, ax = plt.subplots()

ax.plot(
    np.arange(0,N+2),
    np.ones(N+2)*np.mean(ordered_cost_improvements)/1e6,
    label='VoI',
    color='black',
    lw=2.5,
    zorder=1
)
for tech in available_technologies:
    ax.scatter(
        x=np.arange(1,N+1)[ordered_open_techs == tech],
        y=ordered_cost_improvements[ordered_open_techs == tech]/1e6,
        label=tech,
        clip_on=False,
        zorder=10 if tech == 'Li-ion' else 20
    )

plt.xlim(0,N+1)
plt.ylim(0)

plt.xlabel('Ordered Scenarios')
plt.ylabel('Cost Improvement (EUR, millions)')

plt.legend(loc='upper left')
plt.show()